In [1]:
# Installs for the below cells.

# !pip install openai
# !pip install langchain
# !pip install langchain-openai
# !pip install tiktoken


In [2]:
import os

# # Set OPENAI API Key

# os.environ["OPENAI_API_KEY"] = "your openai key"

# OR (load from .env file)
# make sure you have python-dotenv installed
# from dotenv import load_dotenv
# load_dotenv("./.env")

# OpenAI Functions 

Allow you to format the output of an llm to input to a function. Meaning allows you to call functions from outputs of LLMs.

Below is a traditional [openai function for function calling](https://platform.openai.com/docs/guides/function-calling).

In [3]:
import os

In [4]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

We can bind this function to a model using the `.bind` method from the ChatOpenAI object (or any `ChatModel` object in langchain for that matter.)

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


prompt = ChatPromptTemplate.from_messages(
    [
     ("human", "{input}")   
    ]
)

model = ChatOpenAI(temperature=0).bind(functions=functions)

now we can create our runnable with:

In [6]:
# is the same as saying chain = prompt | model
runnable = prompt | model

and we can call it:

In [7]:
runnable.invoke({"input": "What's the weather like in Lisbon?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"LIS"}', 'name': 'weather_search'}}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 66, 'total_tokens': 82}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'function_call', 'logprobs': None}, id='run-897a4a9a-c904-4b90-afad-f5073b6b32d5-0')

Let's update the `functions` variable and add another function.

Let's say search for relevant resources given a query.

In [8]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
    {
      "name": "resources_search",
      "description": "Search for relevant resources given a query",
      "parameters": {
        "type": "object",
        "properties": {
          "query": {
            "type": "string",
            "description": "The query describing the subject of the resources"
          },
        },
        "required": ["query"]
      }
    }
  ]

Now we can take that same model and bind these functions to it:

In [9]:
model = model.bind(functions=functions)

In [10]:
runnable = prompt | model

In [11]:
runnable.invoke({"input": "What are the most useful resources of the langchain Python library?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"langchain Python library"}', 'name': 'resources_search'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 105, 'total_tokens': 122}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'function_call', 'logprobs': None}, id='run-81ffcbce-5804-4a4d-be62-b8022c75bc24-0')

What if we wanted the model to put a json object rather than some string or some object?

In [12]:
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
import json

llm_chat = ChatOpenAI(temperature=0)
output_parser = StrOutputParser()

simple_chain = llm_chat | output_parser | json.loads

simple_chain.invoke("What are fun five names for dogs?The output should be a json object.")

{'names': ['Buddy', 'Luna', 'Max', 'Bailey', 'Coco']}

Below we add a list of other runnables. 

If there is an error it will go through the list of fallback chains (meaning, other ways of processing the input in a useful way.)

In [13]:
final_chain = simple_chain.with_fallbacks([simple_chain])

Now, let's add a prompt template to this, and explore the `.batch` option of a runnable.

In [14]:
from langchain_core.prompts import ChatPromptTemplate

template = "Create a list of 3 exercises to drill this concept: {concept}"

prompt = ChatPromptTemplate.from_template(template)

llm_chat = ChatOpenAI(temperature=0)

output_parser = StrOutputParser()

chain = prompt | llm_chat | output_parser

chain.batch([{"concept": "derivatives"}, {"concept": "linear regression"}])

['1. Find the derivative of the function f(x) = 3x^2 + 5x - 2 using the power rule and the constant multiple rule.\n2. Find the derivative of the function g(x) = sin(x) + cos(x) using the sum rule and the derivative of trigonometric functions.\n3. Find the derivative of the function h(x) = ln(x) + e^x using the derivative of natural logarithm and exponential functions.',
 '1. Simple linear regression practice: Start by generating a dataset with two variables, one independent and one dependent. Use a software tool like Python or R to perform simple linear regression analysis on the dataset and interpret the results.\n\n2. Multiple linear regression practice: Create a dataset with multiple independent variables and one dependent variable. Use a regression tool to perform multiple linear regression analysis and assess the impact of each independent variable on the dependent variable.\n\n3. Residual analysis practice: Generate a dataset with a linear relationship between the independent an